In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [19]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [20]:
df["pickup_hour"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 87620 entries, 0 to 87619
Series name: pickup_hour
Non-Null Count  Dtype         
--------------  -----         
87620 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 684.7 KB


In [21]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(55900, 674)
(55900,)
(31720, 674)
(31720,)


In [22]:
X_train

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2023-01-29,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2023-01-30,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2023-01-31,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2023-02-01,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2023-02-02,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55895,90,64,43,20,13,11,16,29,48,61,...,76,91,97,101,80,94,100,69,2023-08-27,263
55896,18,12,5,5,4,22,48,74,95,109,...,90,80,80,72,91,67,53,26,2023-08-28,263
55897,12,11,10,5,4,15,44,97,118,107,...,59,79,84,69,80,57,37,20,2023-08-29,263
55898,26,11,8,0,8,10,44,97,126,121,...,75,92,91,95,72,68,63,30,2023-08-30,263


In [23]:
def average_rides_last_4_weeks(X: pd.DataFrame) -> pd.DataFrame:
    last_4_weeks_columns = [
            f"rides_t-{7*24}",  # 1 week ago
            f"rides_t-{14*24}", # 2 weeks ago
            f"rides_t-{21*24}", # 3 weeks ago
            f"rides_t-{28*24}"  # 4 weeks ago
        ]

        # Ensure the required columns exist in the test DataFrame
    for col in last_4_weeks_columns:
        if col not in X.columns:
            raise ValueError(f"Missing required column: {col}")

    # Calculate the average of the last 4 weeks
    X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)

    return X

from sklearn.preprocessing import FunctionTransformer

add_feature_average_rides_last_4_weeks = FunctionTransformer(
    average_rides_last_4_weeks, validate=False
)

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin

class TemporalFeatureEngineer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_ = X.copy()
        X_["hour"] = X_["pickup_hour"].dt.hour
        X_["day_of_week"] = X_["pickup_hour"].dt.dayofweek

        return X_.drop(columns=["pickup_hour", "pickup_location_id"])

add_temporal_features = TemporalFeatureEngineer()


In [25]:
import lightgbm as lgb

from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor()
)

In [26]:
features = df.drop(columns=["target"])
target = df["target"]
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

    print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")

Train size: 14605, Test size: 14603
Train size: 29208, Test size: 14603
Train size: 43811, Test size: 14603
Train size: 58414, Test size: 14603
Train size: 73017, Test size: 14603


In [19]:
X_train

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2023-01-29,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2023-01-30,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2023-01-31,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2023-02-01,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2023-02-02,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73012,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,2023-09-06,220
73013,0,0,0,0,0,0,0,1,2,2,...,0,0,0,0,0,0,0,0,2023-09-07,220
73014,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,2023-09-08,220
73015,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,2023-09-09,220


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb

# Define the pipeline
pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,  # Feature engineering step 1
    add_temporal_features,  # Feature engineering step 2
    lgb.LGBMRegressor()  # LightGBM model
)

# Define hyperparameter search space
param_distributions = {
    "lgbmregressor__num_leaves": [20, 31, 50, 100],
    "lgbmregressor__max_depth": [-1, 5, 10, 15],
    "lgbmregressor__learning_rate": [0.01, 0.05, 0.1, 0.2],
    "lgbmregressor__subsample": [0.7, 0.8, 0.9, 1.0]
}

# Create RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    scoring="neg_mean_absolute_error",  # MAE as scoring metric
    n_iter=10,  # Number of random samples
    cv=3,  # 3-fold cross-validation
    verbose=1,
    random_state=42
)

# Train with hyperparameter tuning
random_search.fit(X_train, y_train)

# Print best parameters
print("Best Parameters:", random_search.best_params_)
from sklearn.metrics import mean_absolute_error
best_params = random_search.best_params_
best_model = random_search.best_estimator_

# Predict & compute MAE
y_pred = best_model.predict(X_test)
best_mae = mean_absolute_error(y_test, y_pred)
print(best_mae)


# Optuna hyper parameter tuning

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

# Define a fixed set of hyperparameters (except learning rate)
fixed_params = {
    "num_leaves": 50,
    "max_depth": 10,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 0.1,
    "reg_lambda": 0.1,
    "n_estimators": 200  # A reasonable default
}

# Objective function for learning rate tuning
def tune_learning_rate(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 0.001, 0.3)

    pipeline = make_pipeline(
        add_feature_average_rides_last_4_weeks,
        add_temporal_features,
        lgb.LGBMRegressor(learning_rate=learning_rate, **fixed_params)
    )

    # Train the model
    pipeline.fit(X_train, y_train)

    # Predict and evaluate MAE
    y_pred = pipeline.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

# Optimize learning rate
study_lr = optuna.create_study(direction="minimize")
study_lr.optimize(tune_learning_rate, n_trials=20)  # Try 20 learning rates

best_learning_rate = study_lr.best_params["learning_rate"]
print(f"Best Learning Rate: {best_learning_rate}")

[I 2025-03-03 12:42:47,921] A new study created in memory with name: no-name-1bb3b5ef-48f0-4eff-a054-d8830637128a
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/555391110.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.001, 0.3)
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.066570 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 12:42:55,703] Trial 0 finished with value: 4.203580427580282 and parameters: {'learning_rate': 0.010908981164979825}. Best is trial 0 with value: 4.203580427580282.
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/555391110.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.001, 0.3)
/var/folders/x5/7fm

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073654 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 12:43:03,231] Trial 1 finished with value: 3.9510272371210897 and parameters: {'learning_rate': 0.012285272825043311}. Best is trial 1 with value: 3.9510272371210897.
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/555391110.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.001, 0.3)
/var/folders/x5/7

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/555391110.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 0.001, 0.3)
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.069283 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 12:44:58,530] Trial 19 finished with value: 3.5887826470266924 and parameters: {'learning_rate': 0.017608258814279996}. Best is trial 19 with value: 3.5887826470266924.


Best Learning Rate: 0.017608258814279996


In [28]:
# Objective function for tuning all other hyperparameters
def tune_hyperparameters(trial):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-5, 10),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-5, 10),
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "learning_rate": best_learning_rate  # Use best found learning rate
    }

    pipeline = make_pipeline(
        add_feature_average_rides_last_4_weeks,
        add_temporal_features,
        lgb.LGBMRegressor(**params)
    )

    # Train the model
    pipeline.fit(X_train, y_train)

    # Predict and evaluate MAE
    y_pred = pipeline.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

# Run hyperparameter tuning with Optuna
study_hp = optuna.create_study(direction="minimize")
study_hp.optimize(tune_hyperparameters, n_trials=30)  # Run 30 trials

# Get the best parameters
best_params = study_hp.best_params
print("Best Hyperparameters:", best_params)

# Train final model with best hyperparameters
final_pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor(**best_params)
)

final_pipeline.fit(X_train, y_train)
y_pred_final = final_pipeline.predict(X_test)

# Compute final MAE
final_mae = mean_absolute_error(y_test, y_pred_final)
print(f"Final MAE after tuning: {final_mae}")

[I 2025-03-03 12:48:20,919] A new study created in memory with name: no-name-b69a3130-19ad-4c56-974f-fdd923d365a4
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/4122175033.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/4122175033.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/4122175033.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/op

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.069188 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 12:48:34,767] Trial 0 finished with value: 3.830953272441932 and parameters: {'num_leaves': 95, 'max_depth': 11, 'subsample': 0.7364533352704296, 'colsample_bytree': 0.6925637247977867, 'reg_alpha': 0.32665333982882105, 'reg_lambda': 0.012961366615435201, 'n_estimators': 357}. Best is trial 0 with value: 3.830953272441932.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/4122175033.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/4122175033.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/4122175033.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "reg_alpha": trial.suggest_loguniform

...

/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 12:54:43,286] Trial 28 finished with value: 3.612052109580532 and parameters: {'num_leaves': 188, 'max_depth': 10, 'subsample': 0.7702615920174012, 'colsample_bytree': 0.643482196209823, 'reg_alpha': 0.0034336062977246115, 'reg_lambda': 0.00022155166745181247, 'n_estimators': 284}. Best is trial 9 with value: 3.5202971836836445.
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/4122175033.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optun

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.070918 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 12:54:53,063] Trial 29 finished with value: 3.8239030452394176 and parameters: {'num_leaves': 100, 'max_depth': 9, 'subsample': 0.8480265982027029, 'colsample_bytree': 0.7249340521431494, 'reg_alpha': 0.44029430020074833, 'reg_lambda': 0.01340721541176389, 'n_estimators': 149}. Best is trial 9 with value: 3.5202971836836445.
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

Best Hyperparameters: {'num_leaves': 127, 'max_depth': 10, 'subsample': 0.5082042241739079, 'colsample_bytree': 0.5726683115827711, 'reg_alpha': 2.3580519960053, 'reg_lambda': 0.8441510180435834, 'n_estimators': 252}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.065629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_56242/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)


In [27]:
import optuna
import lightgbm as lgb
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

# Define the objective function
def objective(trial):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "max_depth": trial.suggest_int("max_depth", -1, 15),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.2),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-5, 10),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-5, 10),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500)
    }

    # Define the pipeline with feature engineering and LGBMRegressor
    pipeline = make_pipeline(
        add_feature_average_rides_last_4_weeks,
        add_temporal_features,
        lgb.LGBMRegressor(**params)
    )

    # Train the model
    pipeline.fit(X_train, y_train)

    # Predict on test data
    y_pred = pipeline.predict(X_test)

    # Compute MAE (Lower is better)
    return mean_absolute_error(y_test, y_pred)

# Run optimization with Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)  # Run 30 trials

# Print best parameters
print("Best Hyperparameters:", study.best_params_)

# Train final model with best hyperparameters
best_params = study.best_params_
final_pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor(**best_params)
)

final_pipeline.fit(X_train, y_train)
y_pred_final = final_pipeline.predict(X_test)

# Compute final MAE
final_mae = mean_absolute_error(y_test, y_pred_final)
print(f"Final MAE after tuning: {final_mae}")

[I 2025-03-03 00:58:56,621] A new study created in memory with name: no-name-7f607f64-486b-4d46-9a79-4bb29a553ac2
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/1827688219.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.2),
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/1827688219.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/1827688219.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See http

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073725 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 00:58:59,288] Trial 0 finished with value: 8.480232558747542 and parameters: {'num_leaves': 27, 'max_depth': 11, 'learning_rate': 0.01489726617616917, 'subsample': 0.5157514964387868, 'colsample_bytree': 0.8921081849153656, 'reg_alpha': 0.00017920491815914892, 'reg_lambda': 4.80726198752688e-05, 'n_estimators': 60}. Best is trial 0 with value: 8.480232558747542.
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/1827688219.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed i

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.066191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 00:59:11,477] Trial 1 finished with value: 3.6858701461396595 and parameters: {'num_leaves': 29, 'max_depth': 8, 'learning_rate': 0.014548790475350107, 'subsample': 0.7556013304302056, 'colsample_bytree': 0.5559399123004378, 'reg_alpha': 0.030372961220879967, 'reg_lambda': 3.4563635457854507, 'n_estimators': 391}. Best is trial 1 with value: 3.6858701461396595.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.082110 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 01:02:40,560] Trial 28 finished with value: 3.676040676743624 and parameters: {'num_leaves': 80, 'max_depth': 12, 'learning_rate': 0.040879214657004136, 'subsample': 0.6393194403169666, 'colsample_bytree': 0.9603351239277065, 'reg_alpha': 8.931480963336875e-05, 'reg_lambda': 0.00013937081885831312, 'n_estimators': 95}. Best is trial 6 with value: 3.6110394957102607.
/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/1827688219.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be remov

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.099333 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
[I 2025-03-03 01:02:46,089] Trial 29 finished with value: 3.9925319499848437 and parameters: {'num_leaves': 121, 'max_depth': 15, 'learning_rate': 0.08679210770552659, 'subsample': 0.5186167725341584, 'colsample_bytree': 0.8996393794315982, 'reg_alpha': 0.00034994233012159445, 'reg_lambda': 3.097436000532844e-05, 'n_estimators': 75}. Best is trial 6 with value: 3.6110394957102607.


AttributeError: 'Study' object has no attribute 'best_params_'

In [29]:
# Run optimization with 30 trials
# study.optimize(objective, n_trials=30)  # Run 30 trials

# # Print best parameters
print("Best Hyperparameters:", study.best_params)  # Fixed the attribute name

# Train final model with best hyperparameters
best_params = study.best_params  # Fixed the attribute name
final_pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor(**best_params)
)

final_pipeline.fit(X_train, y_train)
y_pred_final = final_pipeline.predict(X_test)

# Compute final MAE
final_mae = mean_absolute_error(y_test, y_pred_final)
print(f"Final MAE after tuning: {final_mae}")

Best Hyperparameters: {'num_leaves': 131, 'max_depth': -1, 'learning_rate': 0.030068006503670367, 'subsample': 0.5141956093032208, 'colsample_bytree': 0.9227261187950799, 'reg_alpha': 0.0001896433258382781, 'reg_lambda': 0.11301759949439863, 'n_estimators': 129}


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073465 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 161882
[LightGBM] [Info] Number of data points in the train set: 73017, number of used features: 674
[LightGBM] [Info] Start training from score 10.087267
Final MAE after tuning: 3.6110394957102607


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)


In [21]:
from sklearn.metrics import mean_absolute_error
best_params = random_search.best_params_
best_model = random_search.best_estimator_

# Predict & compute MAE
y_pred = best_model.predict(X_test)
best_mae = mean_absolute_error(y_test, y_pred)
print(best_mae)

3.7376018753407343


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_68783/742093930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)


| **Hyperparameter**       | **Description**                                                                                                                                                                                                 | **Impact**                                                                                                                                                                                                                     |
|---------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `num_leaves`             | Maximum number of leaves in one tree. Larger values increase model complexity and accuracy but may lead to overfitting.                                                                                         | High: Directly controls the complexity of the model. Larger values can improve accuracy but increase the risk of overfitting.                                                                                                  |
| `max_depth`              | Maximum depth of a tree. Limits the depth of the tree to prevent overfitting. Use `-1` for no limit.                                                                                                           | High: Limits the model's ability to capture complex patterns. Shallower trees reduce overfitting but may underfit.                                                                                                             |
| `learning_rate`          | Step size for updating weights. Smaller values make training slower but improve generalization.                                                                                                                | High: Affects convergence speed and generalization. Lower values often require more iterations (`n_estimators`).                                                                                                               |
| `n_estimators`           | Number of boosting iterations (trees). Higher values improve accuracy but increase training time.                                                                                                              | High: Controls the number of trees in the ensemble. Too many trees can lead to overfitting if `learning_rate` is not adjusted.                                                                                                |
| `feature_fraction`       | Fraction of features (columns) to randomly select for each tree. Helps prevent overfitting.                                                                                                                    | Medium-High: Reduces overfitting and speeds up training. Lower values may lead to underfitting.                                                                                                                               |
| `bagging_fraction`       | Fraction of data (rows) to randomly select for each iteration. Works with `bagging_freq`.                                                                                                                       | Medium-High: Reduces overfitting and improves generalization. Lower values may lead to underfitting.                                                                                                                           |
| `bagging_freq`           | Frequency of bagging. For example, `5` means bagging is performed every 5 iterations.                                                                                                                          | Medium: Works with `bagging_fraction` to control how often bagging is applied.                                                                                                                                                |
| `min_child_samples`      | Minimum number of data points required in a leaf. Larger values prevent overfitting by limiting leaf size.                                                                                                     | Medium: Controls overfitting by limiting the size of leaf nodes. Higher values may lead to underfitting.                                                                                                                       |
| `colsample_bytree`       | Alias for `feature_fraction`. Fraction of features to randomly select for each tree.                                                                                                                            | Medium: Similar to `feature_fraction`, reduces overfitting and speeds up training.                                                                                                                                            |
| `subsample`              | Alias for `bagging_fraction`. Fraction of data to randomly select for each iteration.                                                                                                                           | Medium: Similar to `bagging_fraction`, reduces overfitting and improves generalization.                                                                                                                                       |
| `reg_alpha`              | L1 regularization term on weights. Adds a penalty for large coefficients to reduce overfitting.                                                                                                                | Medium: Helps reduce overfitting, especially in high-dimensional data.                                                                                                                                                        |
| `reg_lambda`             | L2 regularization term on weights. Adds a penalty for large coefficients to reduce overfitting.                                                                                                                | Medium: Similar to `reg_alpha`, but penalizes squared coefficients.                                                                                                                                                           |
| `max_bin`                | Maximum number of bins for discretizing continuous features. Higher values improve accuracy but increase training time.                                                                                         | Medium: Affects how continuous features are bucketed. Higher values improve precision but increase computational cost.                                                                                                         |
| `min_split_gain`         | Minimum gain required to split a node. Higher values prevent splitting nodes with low information gain.                                                                                                         | Medium-Low: Helps control overfitting by limiting unnecessary splits.                                                                                                                                                         |
| `boosting_type`          | Type of boosting algorithm. Options: `gbdt` (default), `dart`, `goss`.                                                                                                                                          | Medium-Low: Affects the boosting strategy. `dart` and `goss` are alternatives to `gbdt` for specific use cases.                                                                                                               |
| `objective`              | Objective function to optimize. Common options: `regression`, `regression_l1`, `huber`, `fair`.                                                                                                                | Medium-Low: Determines the loss function. Impacts how the model optimizes predictions.                                                                                                                                         |
| `verbosity`              | Controls the level of logging. Higher values provide more detailed logs.                                                                                                                                       | Low: Does not affect model performance but helps with debugging.                                                                                                                                                              |
| `random_state`           | Seed for reproducibility. Ensures consistent results across runs.                                                                                                                                               | Low: Does not affect model performance but ensures reproducibility.                                                                                                                                                           |
| `early_stopping_round`   | Stops training if validation score does not improve for a specified number of rounds.                                                                                                                           | Low: Helps save time during training but does not directly affect model performance.                                                                                                                                           |